<a href="https://colab.research.google.com/github/zheng-zhu-dopay/MyTest/blob/main/GPU_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [4]:
torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:08<00:00, 20.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1), #(i-k+2p)/s + 1
            nn.MaxPool2d(kernel_size=2, stride=2), # (M-P)/s + 1 = 16
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=32*8*8, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = CNN().to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [10]:
#Training
import time
start_time = time.time()

loss_list = []
accuracy_list = []
epoch_size = 20
print('epoch is :{}'.format(epoch_size))
for epoch in range(epoch_size):
    epoch_loss = 0.0
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    loss_list.append(epoch_loss/len(trainloader))

    with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = correct/total
        accuracy_list.append(accuracy)

    print(f"Epoch [{epoch+1}/20] Training Accuracy: {accuracy:.4f}%")

print('Training took {:.2f} seconds'.format(time.time() - start_time))

epoch is :20
Epoch [1/20] Training Accuracy: 0.3153%
Epoch [2/20] Training Accuracy: 0.4048%
Epoch [3/20] Training Accuracy: 0.4237%
Epoch [4/20] Training Accuracy: 0.4859%
Epoch [5/20] Training Accuracy: 0.4968%
Epoch [6/20] Training Accuracy: 0.5133%
Epoch [7/20] Training Accuracy: 0.5405%
Epoch [8/20] Training Accuracy: 0.5653%
Epoch [9/20] Training Accuracy: 0.5559%
Epoch [10/20] Training Accuracy: 0.5837%
Epoch [11/20] Training Accuracy: 0.5811%
Epoch [12/20] Training Accuracy: 0.5958%
Epoch [13/20] Training Accuracy: 0.5910%
Epoch [14/20] Training Accuracy: 0.6243%
Epoch [15/20] Training Accuracy: 0.6362%
Epoch [16/20] Training Accuracy: 0.6239%
Epoch [17/20] Training Accuracy: 0.6569%
Epoch [18/20] Training Accuracy: 0.6370%
Epoch [19/20] Training Accuracy: 0.6070%
Epoch [20/20] Training Accuracy: 0.6355%


In [11]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in testloader:
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    accuracy = correct/total
    accuracy_list.append(accuracy)
  print('Test accuracy: {:.4f}'.format(accuracy))

Test accuracy: 0.6355


In [12]:
save_state = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch}
torch.save(save_state, 'CNN_GPU.pth')

In [13]:
state = torch.load('CNN_GPU.pth')
model.load_state_dict(state['model'])
optimizer.load_state_dict(state['optimizer'])
epoch = state['epoch']

OrderedDict([('conv.0.weight',
              tensor([[[[ 8.4390e-02,  1.2785e-01,  1.9863e-01],
                        [-2.1495e-01, -4.2825e-01, -2.6062e-01],
                        [-2.9940e-01, -2.3997e-01, -2.0484e-01]],
              
                       [[ 1.6104e-01, -1.1640e-02,  1.7379e-01],
                        [-2.6399e-01, -4.3385e-02, -5.6065e-02],
                        [-7.5870e-02, -2.4960e-01, -2.3940e-01]],
              
                       [[ 6.2660e-02,  3.9671e-01,  2.5087e-01],
                        [ 2.4693e-01,  3.2261e-01,  1.8271e-01],
                        [ 4.4273e-02,  1.1985e-01,  2.6017e-01]]],
              
              
                      [[[ 5.1404e-01, -4.1858e-01, -7.1124e-02],
                        [ 1.2107e-02, -6.0369e-01,  2.5554e-01],
                        [-3.8491e-01,  2.7350e-01,  2.7250e-01]],
              
                       [[ 5.7216e-01, -4.7360e-01, -8.7299e-02],
                        [ 9.1652e-02, -4.186